# Problema de negócio

Empresa **STAR JEANS**

Eduardo e Marcelo são dois brasileiros, amigos e sócios de empreendimento. Depois de vários
negócio bem sucedidos, eles estão planejando entrar no mercado de moda dos USA como um
modelo de negócio do tipo E-commerce.

A idéia inicial é entrar no mercado com apenas um produto e para um público específico, no caso
o produto seria calças Jenas para o público masculino. O objetivo é manter o custo de operação
baixo e escalar a medida que forem conseguindo clientes.

Porém, mesmo com o produto de entrada e a audiência definidos, os dois sócios não tem experiência
nesse mercado de moda e portanto não sabem definir coisas básicas como preço, o tipo de calça e
o material para a fabricação de cada peça.

Assim, os dois sócios contrataram uma consultoria de Ciência de Dados para responder as seguintes
perguntas: 

   1. Qual o melhor preço de venda para as calças? 
   2. Quantos tipos de calças e suas cores para o produto inicial? 
   3. Quais as matérias-prima necessárias para confeccionar as calças?

As principais concorrentes da empresa Start Jeans são as americadas H&M e Macys.

# Planejamento para solução

 ## Problema de Negócio
Qual o melhor preço de venda para calças?

## Saída do Projeto ( Produto final )
1. A resposta para a pergunta
    - Mediana dos preços dos concorrentes
 
 
2. Formato da entrega
    - Tabela ou Gráfico
 
 
3. Local da entrega
    - App Streamlit


## Processo ( Passo a Passo )

1. Passo a passso para construir o cálculo da mediana ou média
    - Realizar o calculo da mediana sobre o produto, tipo e cor


2. Definir o formato da entrega ( Visualização, Tabela, Frase )
    - Gráfico de barras com a mediana dos preço dos produtos, por tipo e cor dos últimos 30 dias.
    - Tabela com as seguintes colunas: id | product_name | product_type | product_color | product_price
    - Definição do schema: Colunas e seu tipo
    - Definição a infraestrutura de armazenamento ( SQLITE3 )
    - Design do ETL ( Scripts de Extração, Transformação e Carga )
    - Planejamento de Agendamento dos scripts ( dependencias entre os scripts )
    - Fazer as visualizações
    - Entrega do produto final


3. Decidir o local de entrega ( PowerBi, Telegram, Email, Streamlit, Intranet )
    - App com Streamlit


## Entrada ( Fonte de dados )

### 1. Fonte de dados
- Site da H&M: https://www2.hm.com/en_us/men/products/jeans.html
- Site da Macys: https://www.macys.com/shop/mens-clothing/mens-jeans

### 2. Ferramentas
- Python 3.8.0
- Bibliotecas de Webscrapping ( BS4, Selenium )
- PyCharm
- Jupyter Notebook ( Analise e prototipagens )
- Crontjob, Airflow
- Streamlit

# 1.0. Web scraping

In [2]:
import requests
import pandas as pd
import re

from bs4      import BeautifulSoup
from datetime import datetime

## 1.1. Número Max de páginas

In [2]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text, 'html.parser' )

total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )
total_item = int( total_item )

# round to the top value
page_number = np.ceil( total_item/36 )

url02 = url + '?page-size=' + str( int( page_number*36 ) )
url02

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=72'

## 1.2. Coleta de dados

In [3]:
url = url02

custom_header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
page = requests.get( url, headers=custom_header )

soup = BeautifulSoup( page.text, 'html.parser' )

products = soup.find( 'ul', class_='products-listing small' )
products_list = products.find_all( 'article', class_='hm-product-item' )

#id
product_id = [p.get( 'data-articlecode' ) for p in products_list]

#category
product_cat = [p.get( 'data-category' ) for p in products_list]

# name
product_name = [p.find( 'a', class_='link' ).get_text() for p in products_list]

# price
product_price = [p.find( 'span', class_='price regular' ).get_text() for p in products_list]

# dataframe from the products showed in main page
data = pd.DataFrame( [product_id, product_cat, product_name, product_price] ).T
data.columns = ['id', 'category','name','price']

# creating two differents columns from the ID wich represents a compound information 
# about color and style
data['style_id'] = data['id'].apply( lambda x: x[:-3] )
data['color_id'] = data['id'].apply( lambda x: x[-3:] )

data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )

# general data
data

,id,category,name,price,style_id,color_id,scrapy_datetime
0,0690449036,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,036,2021-12-26 16:35:37
1,0690449051,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,051,2021-12-26 16:35:37
2,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,043,2021-12-26 16:35:37
3,1008549006,men_jeans_regular,Regular Jeans,$ 19.99,1008549,006,2021-12-26 16:35:37
4,0985159001,men_jeans_skinny,Skinny Jeans,$ 19.99,0985159,001,2021-12-26 16:35:37
5,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,022,2021-12-26 16:35:37
6,1008549001,men_jeans_regular,Regular Jeans,$ 19.99,1008549,001,2021-12-26 16:35:37
7,0985197001,men_jeans_slim,Slim Jeans,$ 19.99,0985197,001,2021-12-26 16:35:37
8,0690449056,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,056,2021-12-26 16:35:37
9,0985159005,men_jeans_skinny,Skinny Jeans,$ 19.99,0985159,005,2021-12-26 16:35:37


## 1.3. Color and Composition

In [5]:
custom_header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
df_details = pd.DataFrame()

# unique columns for all products 
aux = []

cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']
df_pattern = pd.DataFrame( columns=cols )

# Using just the id from the data to go through each product and catch each feature inside the page of each product
for i in range( len(data) ):

    # Api Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i,'id'] + '.html'
    
    page = requests.get( url, headers=custom_header )
    
    # Beautifulsoup object
    soup = BeautifulSoup( page.text, 'html.parser' )
    
    # color name and product id
    product_list = soup.find_all( 'a', class_='filter-option miniature' )
    
    color_name = [p.get( 'data-color' ) for p in product_list]
    product_id = [p.get( 'data-articlecode' ) for p in product_list]
    
    # color and id data frame
    df_color = pd.DataFrame( [product_id,color_name] ).T
    df_color.columns = ['product_id','color_name']
    
    # generate style_id e color id
    df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
    df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )
    
    # composition 
    product_composition_list = soup.find_all( 'div', class_='pdp-description-list-item' )
    
    composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]
    
    # rename dataframe
    df_aux = pd.DataFrame( composition ).T
    df_aux.columns = df_aux.iloc[0]
    
    df_aux = df_aux.iloc[1:].fillna( method='ffill' ).reset_index(drop=True)
    
    # garantee the same number of columns
    df_aux = pd.concat( [df_pattern, df_aux], axis=0 )
    
    #generate style id + color id
    df_aux['style_id'] = df_aux['Art. No.'].apply( lambda x: x[:-3] )
    df_aux['color_id'] = df_aux['Art. No.'].apply( lambda x: x[-3:] )
    
    # just to know how many features has in the diferents products
    aux = aux + df_aux.columns.tolist()
    
    # merge
    df_sku = pd.merge( df_color, df_aux[['style_id','Fit','Composition','Size', 'Product safety']], how='left', on='style_id' )
    
    df_details = pd.concat( [df_details, df_sku], axis=0 )
    
df_details = df_details.reset_index(drop=True)
data_raw = pd.merge( data, df_details[['style_id','color_name','Fit','Composition','Size', 'Product safety']], how='left', on='style_id' )

In [12]:
data_raw = data_raw[['id','category','name','price','scrapy_datetime','style_id','color_id','color_name','Fit','Size','Product safety','Composition']]

lw = lambda x: x.lower()

data_raw.columns = list( map( lw, data_raw.columns ) )

In [13]:
# data_raw.to_csv( 'data_raw.csv', index=False )

In [28]:
# difinition to var "cols" above
set(aux)

{'Art. No.',
 'Composition',
 'Fit',
 'More sustainable materials',
 'Product safety',
 'Size',
 'color_id',
 'style_id'}

# 2.0. Manipulation

In [313]:
data = pd.read_csv( 'data_raw.csv' )

In [314]:
data.dtypes

Unnamed: 0          int64
id                  int64
category           object
name               object
price              object
scrapy_datetime    object
style_id            int64
color_id            int64
color_name         object
fit                object
size               object
product safety     object
composition        object
dtype: object

In [315]:
data.isnull().sum()

Unnamed: 0            0
id                    0
category              0
name                  0
price                 0
scrapy_datetime       0
style_id              0
color_id              0
color_name            3
fit                   3
size                807
product safety     2492
composition           3
dtype: int64

## 2.1. Change Types

In [316]:
# id
data['id'] = data['id'].astype( int )

# name
data['name'] = data['name'].apply( lambda x: x.replace(' ','_').lower() )

# price
data['price'] = data['price'].apply( lambda x: x.replace('$ ','') ).astype( float )

# time
data['scrapy_datetime'] = pd.to_datetime( data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S')

# color_id and style_id
data['style_id'] = data['style_id'].astype( int )
data['color_id'] = data['color_id'].astype( int )

## 2.2. Feature engineering

In [317]:
# color_name
data['color_name'] = data['color_name'].apply( lambda x: x.replace(' ','_').lower() if pd.notnull(x) else x )

# fit 
data['fit'] = data['fit'].apply( lambda x: x.replace(' ','_').lower() if pd.notnull(x) else x )

# size numeber
data['size_number'] = data['size'].apply( lambda x: re.search( '\d{3}cm', x ).group() if pd.notnull(x) else x )
data['size_number'] = data['size_number'].apply( lambda x: str( x ).replace( 'cm', '') )

# size_model
data['size_model'] = data['size'].str.extract( '(\d+/\d+)' )

# product safety dropped
data.drop( 'product safety', axis=1, inplace=True )

In [318]:
data['composition'].unique()

array(['Cotton 98%, Spandex 2%',
       'Pocket lining: Polyester 65%, Cotton 35%',
       'Lining: Polyester 100%', 'Pocket: Cotton 100%',
       'Shell: Cotton 99%, Spandex 1%', 'Shell: Cotton 98%, Spandex 2%',
       'Pocket lining: Cotton 100%', 'Cotton 99%, Spandex 1%',
       'Shell: Cotton 100%', 'Cotton 100%',
       'Cotton 89%, Polyester 10%, Spandex 1%',
       'Cotton 78%, Polyester 21%, Spandex 1%',
       'Cotton 79%, Polyester 20%, Spandex 1%',
       'Cotton 77%, Polyester 21%, Spandex 2%',
       'Cotton 79%, Polyester 19%, Spandex 2%',
       'Shell: Cotton 90%, Elasterell-P 8%, Spandex 2%',
       'Cotton 90%, Elasterell-P 8%, Spandex 2%', nan,
       'Cotton 93%, Polyester 6%, Spandex 1%'], dtype=object)

Every Pocket lining, Pocket, Lining and shell was dropped.

In [319]:
data = data[~data['composition'].str.contains( 'Pocket lining:', na=False)]
data = data[~data['composition'].str.contains( 'Shell:', na=False)]
data = data[~data['composition'].str.contains( 'Lining:', na=False)]
data = data[~data['composition'].str.contains( 'Pocket:', na=False)]

data = data.drop_duplicates( subset=['id', 'category', 'name', 'price', 'scrapy_datetime',
       'style_id', 'color_id', 'color_name', 'fit'], keep='last' )
data = data.reset_index( drop=True )

In [320]:
data['composition'].unique()

array(['Cotton 98%, Spandex 2%', 'Cotton 99%, Spandex 1%', 'Cotton 100%',
       'Cotton 89%, Polyester 10%, Spandex 1%',
       'Cotton 78%, Polyester 21%, Spandex 1%',
       'Cotton 79%, Polyester 19%, Spandex 2%',
       'Cotton 90%, Elasterell-P 8%, Spandex 2%', nan,
       'Cotton 93%, Polyester 6%, Spandex 1%',
       'Cotton 77%, Polyester 21%, Spandex 2%'], dtype=object)

In [321]:
df = data['composition'].str.split( ',', expand=True )
df

,0,1,2
0,Cotton 98%,Spandex 2%,None
1,Cotton 98%,Spandex 2%,None
2,Cotton 98%,Spandex 2%,None
3,Cotton 98%,Spandex 2%,None
4,Cotton 98%,Spandex 2%,None
...,...,...,...
261,Cotton 100%,None,None
262,Cotton 100%,None,None
263,Cotton 100%,None,None
264,Cotton 100%,None,None


In [322]:
df[0].unique()

array(['Cotton 98%', 'Cotton 99%', 'Cotton 100%', 'Cotton 89%',
       'Cotton 78%', 'Cotton 79%', 'Cotton 90%', nan, 'Cotton 93%',
       'Cotton 77%'], dtype=object)

In [323]:
df[1].unique()

array([' Spandex 2%', ' Spandex 1%', None, ' Polyester 10%',
       ' Polyester 21%', ' Polyester 19%', ' Elasterell-P 8%', nan,
       ' Polyester 6%'], dtype=object)

In [324]:
df[2].unique()

array([None, ' Spandex 1%', ' Spandex 2%', nan], dtype=object)


## COTTON │ POLYESTER │  ELASTERELL │SPANDEX

In [325]:
df_ref = pd.DataFrame( columns=['cotton','polyester','elasterell','spandex'], index=np.arange( len(data ) ) )

# cotton
df_cotton = df[0]
df_cotton.name = 'cotton'

df_ref = pd.concat( [df_ref, df_cotton], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last' )]
df_ref['cotton'] = df_ref['cotton'].fillna( 'Cotton %0' )

# polyester
df_polyester = df.loc[df[1].str.contains( 'Polyester', na=True ), 1]
df_polyester.name = 'polyester'

df_ref = pd.concat( [df_ref, df_polyester], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]
df_ref['polyester'] = df_ref['polyester'].fillna( 'polyester %0')

# elasterell
df_elasterell = df.loc[df[1].str.contains( 'Elasterell', na=True ), 1]
df_elasterell.name = 'elasterell'

df_ref = pd.concat( [df_ref, df_elasterell], axis=1 )
df_ref = df_ref.loc[:, ~df_ref.columns.duplicated( keep='last' ) ]
df_ref['elasterell'] = df_ref['elasterell'].fillna( 'elasterell %0')

# spandex
df_spandex = df.loc[df[1].str.contains( 'Spandex', na=True ), 1]
df_spandex.name = 'spandex'

df_spandex = df_spandex.combine_first( df[2] )
                                                 
df_ref = pd.concat( [df_ref, df_spandex], axis=1 )
df_ref = df_ref.loc[:, ~df_ref.columns.duplicated( keep='last' )]
df_ref['spandex'] = df_ref['spandex'].fillna( 'spandex %0')

# concat com os valores iniciais
data = pd.concat( [data, df_ref], axis=1 )

data['cotton']     = data['cotton'].apply( lambda x: re.search( '\d+',x ).group() if pd.notnull( x ) else x )
data['polyester']  = data['polyester'].apply( lambda x: re.search( '\d+',x ).group() if pd.notnull( x ) else x )
data['elasterell'] = data['elasterell'].apply( lambda x: re.search( '\d+',x ).group() if pd.notnull( x ) else x )
data['spandex']    = data['spandex'].apply( lambda x: re.search( '\d+',x ).group() if pd.notnull( x ) else x )

# drop columns
data = data.drop( columns=['size', 'composition', 'Unnamed: 0'], axis=1 )

# drop duplicatates
data = data.drop_duplicates()

In [327]:
# data.to_csv( 'final_data.csv', index=False )

In [328]:
data = pd.read_csv( 'final_data.csv' )
data.sample(3)

,id,category,name,price,scrapy_datetime,style_id,color_id,color_name,fit,size_number,size_model,cotton,polyester,elasterell,spandex
224,730863033,men_jeans_skinny,skinny_jeans,29.99,2021-12-26 16:35:37,730863,33,dark_blue,skinny_fit,NaN,NaN,98,0,0,2
191,811993036,men_jeans_regular,regular_jeans,29.99,2021-12-26 16:35:37,811993,36,black,regular_fit,182.0,31/32,99,0,0,1
218,1004199003,men_jeans_skinny,skinny_cropped_jeans,29.99,2021-12-26 16:35:37,1004199,3,denim_blue,skinny_fit,187.0,NaN,99,0,0,1
